In [348]:
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd 

In [349]:
# read csv file using pandas
df1 = pd.read_csv('final_Movies_data0.csv')
df2 = pd.read_csv('Translated_telugu_dataset .csv')

df2.rename(columns = {'సినిమా పేరు':'Title', 'దర్శకుడు':'Director', 'నటులు':'Cast', 'శైలి':'Genre', 'తేదీ':'Date','సంవత్సరం':'Year','నెల':'Month','భాష':'Language','Soup/సూప్':'Soup' }, inplace = True)
df1.rename(columns = {'title/الفيلم':'Title','cast/تمثيل':'Cast','genres/النوع':'Genre','Year/السنة':'Year','original_language/اللغة':'Language',},inplace = True)

In [350]:
# remove brackets
df1['Cast'] = df1['Cast'].str.strip('[]')
df1['Genre'] = df1['Genre'].str.strip('[]')

# remove quotes
df1["Cast"] = df1["Cast"].str.replace(r"^'|,'$", "").str.replace("'", '')
df1["Genre"] = df1["Genre"].str.replace(r"^'|,'$", "").str.replace("'", '')

# inorder to add year into soup we need to conevrt into string formte
df1['Year']= df1['Year'].apply(str)

df1.head()

/var/folders/ky/f7y74l5j3qg23vm3ysrdy97m0000gn/T/ipykernel_50674/3738292423.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df1["Cast"] = df1["Cast"].str.replace(r"^'|,'$", "").str.replace("'", '')
/var/folders/ky/f7y74l5j3qg23vm3ysrdy97m0000gn/T/ipykernel_50674/3738292423.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df1["Genre"] = df1["Genre"].str.replace(r"^'|,'$", "").str.replace("'", '')


Language                       Genre  \
0       en   Animation, Comedy, Family   
1       en  Adventure, Fantasy, Family   
2       en             Romance, Comedy   
3       en      Comedy, Drama, Romance   
4       en                      Comedy   

                                       overview/ملخص  \
0  Led by Woody, Andy's toys live happily in his ...   
1  When siblings Judy and Peter discover an encha...   
2  A family wedding reignites the ancient feud be...   
3  Cheated on, mistreated and stepped on, the wom...   
4  Just when George Banks has recovered from his ...   

                                                Cast  \
0  Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...   
1  Robin Williams, Jonathan Hyde, Kirsten Dunst, ...   
2  Walter Matthau, Jack Lemmon, Ann-Margret, Soph...   
3  Whitney Houston, Angela Bassett, Loretta Devin...   
4  Steve Martin, Diane Keaton, Martin Short, Kimb...   

                         Title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                                soup  Year  
0  Tom Hanks Tim Allen Don Rickles Jim Varney Wal...  1995  
1  Robin Williams Jonathan Hyde Kirsten Dunst Bra...  1995  
2  Walter Matthau Jack Lemmon Ann-Margret Sophia ...  1995  
3  Whitney Houston Angela Bassett Loretta Devine ...  1995  
4  Steve Martin Diane Keaton Martin Short Kimberl...  1995

In [351]:
df1 = df1[~df1.Title.isin(df1[df1.duplicated(subset=['Title'])].Title.unique())].reset_index(drop=True)
df2 = df2[~df2.Title.isin(df2[df2.duplicated(subset=['Title'])].Title.unique())].reset_index(drop=True)

In [352]:
data_ref = pd.concat([df1[['Title', 'Cast', 'Year', 'Genre']], df2[['Title', 'Cast', 'Year', 'Genre']]]).dropna()
data_ref

Title  \
0                       Toy Story   
1                         Jumanji   
2                Grumpier Old Men   
3               Waiting to Exhale   
4     Father of the Bride Part II   
...                           ...   
3343                    ఉందా లేదా   
3344        మిడిల్ క్లాసు అబ్బాయి   
3345                          హలో   
3346                   ఒక్క క్షణం   
3347                     2 దేశాలు   

                                                   Cast  Year  \
0     Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...  1995   
1     Robin Williams, Jonathan Hyde, Kirsten Dunst, ...  1995   
2     Walter Matthau, Jack Lemmon, Ann-Margret, Soph...  1995   
3     Whitney Houston, Angela Bassett, Loretta Devin...  1995   
4     Steve Martin, Diane Keaton, Martin Short, Kimb...  1995   
...                                                 ...   ...   
3343                                    రామకృష్ణ, అంకిత  2017   
3344                  నాని, సాయి పల్లవి, Bhumika Chawla  2017   
3345  అఖిల్ అక్కినేని, కళ్యాణి ప్రియదర్శన్, జగపతి బా...  2017   
3346  అల్లు శిరీష్, సురభి, సీరత్ కపూర్, శ్రీనివాస్ అ...  2017   
3347                 సునీల్, మనీషా రాజ్, సంజాజన్, నరేష్  2017   

                           Genre  
0      Animation, Comedy, Family  
1     Adventure, Fantasy, Family  
2                Romance, Comedy  
3         Comedy, Drama, Romance  
4                         Comedy  
...                          ...  
3343                   థ్రిల్లర్  
3344            రొమాంటిక్ డ్రామా  
3345                        చర్య  
3346    సైన్స్ ఫిక్షన్ థ్రిల్లర్  
3347            రొమాంటిక్ కామెడీ  

[14119 rows x 4 columns]

In [353]:
df1['Cast'] = df1['Cast'].str.replace(" ", '')

# create the new soup
df1['Soup'] = df1['Cast']+ '  ' + df1['Genre']+ '  ' + '  ' + df1['Language']

# Lower casing
df1['Soup'] = df1['Soup'].apply(lambda x: ' '.join(x.lower() for x in x.split()))

# remove comma from soup
# df1['Soup'] = df1['Soup'].str.replace(',', '')
df2['Soup'] = df2['Cast']+ '  ' + df2['Genre']+ '  ' + '  ' + 'te'

In [354]:
data = pd.concat([df1[['Soup']], df2[['Soup']]]).dropna()
data

Soup
0     tomhanks,timallen,donrickles,jimvarney,wallace...
1     robinwilliams,jonathanhyde,kirstendunst,bradle...
2     waltermatthau,jacklemmon,ann-margret,sophialor...
3     whitneyhouston,angelabassett,lorettadevine,lel...
4     stevemartin,dianekeaton,martinshort,kimberlywi...
...                                                 ...
3343                   రామకృష్ణ, అంకిత  థ్రిల్లర్    te
3344  నాని, సాయి పల్లవి, Bhumika Chawla  రొమాంటిక్ డ...
3345  అఖిల్ అక్కినేని, కళ్యాణి ప్రియదర్శన్, జగపతి బా...
3346  అల్లు శిరీష్, సురభి, సీరత్ కపూర్, శ్రీనివాస్ అ...
3347  సునీల్, మనీషా రాజ్, సంజాజన్, నరేష్  రొమాంటిక్ ...

[14119 rows x 1 columns]

In [355]:
#Getting the user's input for genre, actors and directors of their liking.


def get_directors():
    #/يوجد ممثل معين؟
    directors = input("Directors Name/మీరు ఎవరి దర్శకుడి సినిమాని చూడటానికి ఆసక్తిగా ఉన్నారు?/يوجد ممثل معين؟")
    directors = " ".join(["".join(n.split()) for n in directors.lower().split(',')])
    return directors

def get_actors():
    #/ما هو نوع الفيلم؟
    actors = input("Whose actor movie you wanna watch?/మీరు ఎవరి నటుడి సినిమాని చూడటానికి ఆసక్తిగా ఉన్నారు?/ما هو نوع الفيلم؟")
    actors = " ".join(["".join(n.split()) for n in actors.lower().split(',')])
    return actors

def get_genres():
    #/مخرج معيّن؟
    genres = input("what kind of Genere you want?/మీకు ఏ సినిమా జానర్‌పై ఆసక్తి ఉంది?/مخرج معيّن؟")
    genres = " ".join(["".join(n.split()) for n in genres.lower().split(',')])
    return genres

def get_Year():
    #/السنة؟
    Year = input("which year movie?/ఏ సంవత్సరం సినిమాలు చూడాలి అనుకుంటున్నారు?/السنة")
    Year = " ".join(["".join(n.split()) for n in Year.lower().split(',')])
    return Year

def get_Language():
    #/ماهي اللغة؟
    Language = input("Entre the Language/ఏ భాష సినిమాలు చూడాలి అనుకుంటున్నారు?/ماهي اللغة؟")
    Language = " ".join(["".join(n.split()) for n in Language.lower().split(',')])
    return Language



def get_searchTerms():
    searchTerms = [] 
        
    directors = get_directors()
    if directors != 'Skip/దాటవేయండి/تخطي':
        searchTerms.append(directors)
        
    actors = get_actors()
    if actors != 'Skip/దాటవేయండి/تخطي':
        searchTerms.append(actors)
        
    genres = get_genres()
    if genres != 'Skip/దాటవేయండి/تخطي':
        searchTerms.append(genres)

        
    Year = get_Year()
    if Year != 'skip/దాటవేయండి/تخطي':
        searchTerms.append(Year)
        
    Language = get_Language()
    if Language != 'Skip/దాటవేయండి/تخطي':
        searchTerms.append(Language)
        
    return searchTerms

In [356]:
count = TfidfVectorizer(stop_words='english')
count_matrix1 = count.fit_transform(data['Soup'])

In [357]:
# searchTerms = ' '.join(get_searchTerms())

In [358]:
searchTerms = 'tomhanks romance en'

In [359]:
count_matrix2 = count.transform([searchTerms])

In [360]:
cosine_sim2 = cosine_similarity(count_matrix1, count_matrix2)
sorted(list(enumerate(cosine_sim2)), key=lambda x: x[1], reverse=True)

[(3075, array([0.27424785])),
 (2170, array([0.27021344])),
 (4889, array([0.25541627])),
 (6584, array([0.24825579])),
 (1874, array([0.2402671])),
 (2176, array([0.23360262])),
 (7792, array([0.23089889])),
 (2512, array([0.22097524])),
 (0, array([0.22065263])),
 (6610, array([0.21924698])),
 (8598, array([0.21356514])),
 (4776, array([0.20504544])),
 (568, array([0.20423318])),
 (592, array([0.20423318])),
 (628, array([0.20423318])),
 (638, array([0.20423318])),
 (712, array([0.20423318])),
 (716, array([0.20423318])),
 (728, array([0.20423318])),
 (782, array([0.20423318])),
 (1031, array([0.20423318])),
 (1195, array([0.20423318])),
 (1316, array([0.20423318])),
 (1395, array([0.20423318])),
 (1527, array([0.20423318])),
 (1533, array([0.20423318])),
 (1554, array([0.20423318])),
 (1663, array([0.20423318])),
 (1685, array([0.20423318])),
 (2229, array([0.20423318])),
 (2465, array([0.20423318])),
 (2539, array([0.20423318])),
 (3726, array([0.20423318])),
 (3879, array([0.20423

In [362]:
idx = [i[0] for i in sorted(list(enumerate(cosine_sim2)), key=lambda x: x[1], reverse=True)][:5]

In [363]:
[[data_ref.iloc[i]['Title']+' | '+data_ref.iloc[i]['Year']+' | '+data_ref.iloc[i]['Genre']+' | '+data_ref.iloc[i]['Cast']] for i in idx]


[['Volunteers | 1985 | Comedy | Tom Hanks, John Candy, Rita Wilson, Tim Thomerson, Gedde Watanabe, George Plimpton, Ernest Harada, Allan Arbus, Xander Berkeley'],
 ['Nothing in Common | 1986 | Drama, Comedy, Romance | Tom Hanks, Jackie Gleason, Eva Marie Saint, Héctor Elizondo, Barry Corbin, Sela Ward, Bess Armstrong, Dan Castellaneta, Mike Hagerty, Tracy Reiner, Johnny Yune, Conrad Janis'],
 ['Joe Versus the Volcano | 1990 | Fantasy, Comedy, Science Fiction, Romance | Tom Hanks, Meg Ryan, Lloyd Bridges, Dan Hedaya, Ossie Davis, Barry McGovern, Robert Stack, Amanda Plummer, Lala Sloatman, Abe Vigoda, Dan Hedaya, Nathan Lane, Carol Kane'],
 ['Dragnet | 1987 | Action, Comedy, Crime | Dan Aykroyd, Tom Hanks, Christopher Plummer, Harry Morgan, Alexandra Paul, "Jack OHalloran", Elizabeth Ashley, Dabney Coleman, Kathleen Freeman, Shannon Tweed, Lisa London'],
 ['Splash | 1984 | Comedy, Fantasy, Romance | Tom Hanks, Daryl Hannah, Eugene Levy, John Candy, Dody Goodman, Shecky Greene, Richard B

In [81]:
df1.iloc[3300]

Language                                                        en
Genre                                    Animation, Comedy, Family
overview/ملخص    Interviews with the public are used to put wor...
Cast                                                              
Title                                            Creature Comforts
soup                                       Animation Comedy Family
Year                                                          1989
Name: 3300, dtype: object

In [ ]:


def make_recommendation(data = data):
    new_row = df.iloc[-1,:].copy() #creating a copy of the last row of the 
    #dfset, which we will use to input the user's input
    
    #grabbing the new wordsoup from the user
    searchTerms = get_searchTerms()    
    new_row.iloc[-1] = " ".join(searchTerms) #adding the input to our new row
    
    #adding the new row to the dfset
    data = data.append(new_row)
    
    #Vectorizing the entire matrix as described above!
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(df['Soup'])

    #running pairwise cosine similarity 
    cosine_sim2 = cosine_similarity(count_matrix, count_matrix) #getting a similarity matrix
    
    #sorting cosine similarities by highest to lowest
    sim_scores = list(enumerate(cosine_sim2[-1,:]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    #matching the similarities to the movie titles and ids
    ranked_titles = []
    for i in range(1, 5):
        indx = sim_scores[i][0]
        ranked_titles.append([df['Title'].iloc[indx],df['Cast'].iloc[indx], df['Year'].iloc[indx], df['Genre'].iloc[indx]])
    
    return ranked_titles

#let's try our recommendation function now
make_recommendation()

In [13]:
import pandas as pd
# df = pd.read_csv('Final_Movies_Multilanguages.csv')
df = pd.read_excel('Final_3_languages.xlsx')


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Getting the user's input for genre, actors and directors of their liking.


def get_directors():
    #/يوجد ممثل معين؟
    directors = input("Directors Name/మీరు ఎవరి దర్శకుడి సినిమాని చూడటానికి ఆసక్తిగా ఉన్నారు?/يوجد ممثل معين؟")
    directors = " ".join(["".join(n.split()) for n in directors.lower().split(',')])
    return directors

def get_actors():
    #/ما هو نوع الفيلم؟
    actors = input("Whose actor movie you wanna watch?/మీరు ఎవరి నటుడి సినిమాని చూడటానికి ఆసక్తిగా ఉన్నారు?/ما هو نوع الفيلم؟")
    actors = " ".join(["".join(n.split()) for n in actors.lower().split(',')])
    return actors

def get_genres():
    #/مخرج معيّن؟
    genres = input("what kind of Genere you want?/మీకు ఏ సినిమా జానర్‌పై ఆసక్తి ఉంది?/مخرج معيّن؟")
    genres = " ".join(["".join(n.split()) for n in genres.lower().split(',')])
    return genres

def get_Year():
    #/السنة؟
    Year = input("which year movie?/ఏ సంవత్సరం సినిమాలు చూడాలి అనుకుంటున్నారు?/السنة")
    Year = " ".join(["".join(n.split()) for n in Year.lower().split(',')])
    return Year

def get_Language():
    #/ماهي اللغة؟
    Language = input("Entre the Language/ఏ భాష సినిమాలు చూడాలి అనుకుంటున్నారు?/ماهي اللغة؟")
    Language = " ".join(["".join(n.split()) for n in Language.lower().split(',')])
    return Language



def get_searchTerms():
    searchTerms = [] 
    
    directors = get_directors()
    if directors != 'Skip/దాటవేయండి/تخطي':
        searchTerms.append(directors)
    
    actors = get_actors()
    if actors != 'Skip/దాటవేయండి/تخطي':
        searchTerms.append(actors)
    
    genres = get_genres()
    if genres != 'Skip/దాటవేయండి/تخطي':
        searchTerms.append(genres)

    
    Year = get_Year()
    if Year != 'skip/దాటవేయండి/تخطي':
        searchTerms.append(Year)
    
    Language = get_Language()
    if Language != 'Skip/దాటవేయండి/تخطي':
        searchTerms.append(Language)
    
    return searchTerms

def make_recommendation(data = df):
    new_row = df.iloc[-1,:].copy() #creating a copy of the last row of the 
    #dfset, which we will use to input the user's input

    #grabbing the new wordsoup from the user
    searchTerms = get_searchTerms()  
    new_row.iloc[-1] = " ".join(searchTerms) #adding the input to our new row

    #adding the new row to the dfset
    data = data.append(new_row)

    #Vectorizing the entire matrix as described above!
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(df['Soup'])

    #running pairwise cosine similarity 
    cosine_sim2 = cosine_similarity(count_matrix, count_matrix) #getting a similarity matrix

    #sorting cosine similarities by highest to lowest
    sim_scores = list(enumerate(cosine_sim2[-1,:]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    #matching the similarities to the movie titles and ids
    ranked_titles = []
    for i in range(1, 5):
        indx = sim_scores[i][0]
        ranked_titles.append([df['Title'].iloc[indx],df['Cast'].iloc[indx], df['Year'].iloc[indx], df['Genre'].iloc[indx]])

    return ranked_titles

#let's try our recommendation function now
make_recommendation()

Directors Name/మీరు ఎవరి దర్శకుడి సినిమాని చూడటానికి ఆసక్తిగా ఉన్నారు?/يوجد ممثل معين؟
Whose actor movie you wanna watch?/మీరు ఎవరి నటుడి సినిమాని చూడటానికి ఆసక్తిగా ఉన్నారు?/ما هو نوع الفيلم؟tom hanks
what kind of Genere you want?/మీకు ఏ సినిమా జానర్‌పై ఆసక్తి ఉంది?/مخرج معيّن؟
which year movie?/ఏ సంవత్సరం సినిమాలు చూడాలి అనుకుంటున్నారు?/السنة
Entre the Language/ఏ భాష సినిమాలు చూడాలి అనుకుంటున్నారు?/ماهي اللغة؟en


[['à°šà°¿à°¤à±\x8dà°°à°¾à°‚à°—à°¦',
  'à°…à°‚à°œà°²à°¿,à°¸à°¾à°•à±\x8dà°·à°¿à°—à±\x81à°²à°¾à°Ÿà°¿,à°¸à°ªà±\x8dà°¤à°—à°¿à°°à°¿,à°°à°¾à°œà°¾à°°à°µà±€à°‚à°¦à±\x8dà°°',
  2017,
  'à°•à°¾à°®à±†à°¡à±€à°¥à±\x8dà°°à°¿à°²à±\x8dà°²à°°à±\x8d'],
 ['à°’à°•à±\x8dà°•à°¡à±\x81 à°®à°¿à°—à°¿à°²à°¾à°¡à±\x81',
  'à°®à°‚à°šà±\x81à°®à°¨à±‹à°œà±\x8d,à°…à°¨à°¿à°·à°¾à°…à°‚à°¬à±\x8dà°°à±‹à°¸à±\x8d,à°¸à±\x81à°¹à°¾à°¸à°¿à°¨à°¿à°®à°£à°¿à°°à°¤à±\x8dà°¨à°‚,à°®à°¿à°²à°¿à°‚à°¦à±\x8dà°—à±\x81à°£à°¾à°œà±€,à°ªà±‹à°¸à°¾à°¨à°¿à°•à±ƒà°·à±\x8dà°£à°®à±\x81à°°à°³à°¿',
  2017,
  'à°¥à±\x8dà°°à°¿à°²à±\x8dà°²à°°à±\x8d/à°¯à°¾à°•à±\x8dà°·à°¨à±\x8d'],
 ['VIP 2',
  'à°§à°¨à±\x81à°·à±\x8d,à°•à°¾à°œà°²à±\x8d,à°…à°®à°²à°¾à°ªà°¾à°²à±\x8d',
  2017,
  'à°®à°¸à°¾à°²à°¾'],
 ['à°•à±\x81à°Ÿà±\x81à°‚à°¬ à°•à°¥à°¾ à°šà°¿à°¤à±\x8dà°°à°‚',
  'à°¨à°‚à°¦à±\x81,à°¶à±\x8dà°°à±€à°®à±\x81à°–à°¿,à°•à°®à°²à±\x8dà°•à°¾à°®à°°à°¾à°œà±\x81',
  2017,
  'à°°à±Šà°®à°¾à°‚à°Ÿà°¿à°•à±\x8dà°¥à±\x8dà°°à°¿à°²à±\x8dà°²à°°à±\x8d']]

# Below things need do

In [ ]:
# Lower casing 
data_full['soup'] = data_full['soup'].apply(lambda x: ' '.join(x.lower() for x in x.split()))

In [ ]:
import nltk
# let's see the 10 most words used:
freq = nltk.FreqDist(' '.join(data_full['soup']).split())

freq.most_common(10)

In [ ]:
from nltk import FreqDist
# 10 least used words:
last_ten = FreqDist(dict(freq.most_common()[-10:]))
last_ten

last_ten.plot()

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(width=800, height=600, max_words=200).generate_from_frequencies(freq)





plt.figure(figsize=(8, 8))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

#chatbot 
https://youtu.be/1lwddP0KUEg

https://youtu.be/XbLLwN61Upw